# Fine-tuning main class model

First load the model and define the file paths for the *mapping* dicts. The mapping is used to remap the labeled masks for the specific models (in the *hierarchy*).

Load the main class model, for C, D, and E.

|Class|Description|
|---|---|
|C|Woods & forests|
|D|Moor & heath|
|E|Agro-pastoral land|

In [ ]:
import sys
import os
sys.path.append('../cnn-land-cover/scripts')
import land_cover_models as lcm
import land_cover_visualisation as lcv
import pytorch_lightning as pl
import pytorch_lightning.loggers as loggers
import torch
import pickle
import glob
import utils

In [ ]:
# Set constants
LOG_DIR = './logs'
BATCH_SIZE = 15
torch.set_float32_matmul_precision('high')  # enable tensor cores on GPU
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
MD_PATHS = utils.MD_PATHS

# Check available GPUs and set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device:", device)

In [ ]:
# Load Base Model 

def load_model_variant(name, n_classes, mapping_key, model_folder, model_filename):
    # Load pretrained LCM model
    m_variant = lcm.load_model_auto(folder=model_folder, filename=model_filename)
    
    # Initialize base model with specified parameters
    base_model = lcm.LandCoverUNet(
        n_classes=n_classes,
        encoder_name='resnet50',
        loss_function='cross_entropy',
        skip_factor_eval=1,
        first_class_is_no_class=False
    )
    base_model.dict_training_details['class_name_list'] = m_variant.dict_training_details['class_name_list']
    
    # Load specific mapping dictionary
    with open(MD_PATHS[mapping_key], 'rb') as f:
        mapping_dict = pickle.load(f)
    return base_model, mapping_dict

In [ ]:
# Data Loader

def setup_dataloaders(data_key, preprocess_func, batch_size=BATCH_SIZE):
    # Define data paths
    DATA = '../'
    DATA_TRN_M = os.path.join(DATA, 'pd_lc_eco_rgb_patches_512_train')
    DATA_TST_M = os.path.join(DATA, 'pd_lc_eco_rgb_patches_512_test')
    
    # Setup data loaders
    dataloaders = {
        'train_m': torch.utils.data.DataLoader(
            utils.load_patches_dataset(DATA_TRN_M, MD_PATHS[data_key], proc_fn=preprocess_func),
            batch_size=batch_size,
            num_workers=os.cpu_count() - 1,
            shuffle=True
        ),
        'test_m': torch.utils.data.DataLoader(
            utils.load_patches_dataset(DATA_TST_M, MD_PATHS[data_key], proc_fn=preprocess_func),
            batch_size=batch_size,
            num_workers=os.cpu_count() - 1,
            shuffle=False
        )
    }
    return dataloaders

In [ ]:
# Trainer

def train_and_evaluate_model(base_model, dataloaders, log_name, max_epochs=60):
    # Set up logger for TensorBoard
    logger = loggers.TensorBoardLogger(LOG_DIR, name=log_name)
    
    # Trainer configuration
    trainer = pl.Trainer(
        max_epochs=max_epochs,
        logger=logger,
        log_every_n_steps=20,
        precision='16-mixed',
        accelerator='gpu' if torch.cuda.is_available() else 'cpu',
        devices=1
    )
    
    # Fit the model
    trainer.fit(model=base_model, train_dataloaders=dataloaders['train_m'], val_dataloaders=dataloaders['test_m'])
    
    # Test the model
    trainer.test(base_model, dataloaders=dataloaders['test_m'])
    
    return base_model

In [ ]:
# CM Plotting Function

def plot_confusion_matrix(base_model, title='Original test samples'):
    cm = lcv.plot_confusion_summary(
        model=base_model,
        conf_mat=None,
        class_name_list=base_model.dict_training_details['class_name_list'],
        normalise_hm=True,
        skip_factor=1,
        fmt_annot='.1f',
        print_main_text=False,
        suppress_zero_annot=True,
        remove_no_class_if_present=True,
        class_indices_to_remove=[],
        title_hm=title
    )

# Model execute

In [ ]:
# Execute Main Model

# Load model variant
base_model, mapping_dict = load_model_variant(
    name='Main',
    n_classes=4,
    mapping_key='main',
    model_folder='../models/orig',
    model_filename='main_LCU_2023-04-24-1259.pth'
)

# Set up dataloaders
dataloaders = setup_dataloaders(data_key='main', preprocess_func=base_model.preprocessing_func)

# Train and evaluate model
base_model = train_and_evaluate_model(base_model, dataloaders, log_name='DA_main_v1')

# Plot confusion matrix
plot_confusion_matrix(base_model, title='Original test samples for Main')

In [ ]:
# Execute Model C

base_model, mapping_dict = load_model_variant(
    name='C',
    n_classes=4,
    mapping_key='c',
    model_folder='../models',
    model_filename='C_LCU_2023-04-21-1335.pth' # fix this
)

dataloaders = setup_dataloaders(data_key='c', preprocess_func=base_model.preprocessing_func)
base_model = train_and_evaluate_model(base_model, dataloaders, log_name='DA_C')
plot_confusion_matrix(base_model, title='Original test samples for Model C')
torch.cuda.empty_cache()  # Clear GPU memory

In [ ]:
# Execute Model D

base_model, mapping_dict = load_model_variant(
    name='D',
    n_classes=4,
    mapping_key='d',
    model_folder='../models',
    model_filename='C_LCU_2023-04-21-1335.pth' # fix this
)

dataloaders = setup_dataloaders(data_key='c', preprocess_func=base_model.preprocessing_func)
base_model = train_and_evaluate_model(base_model, dataloaders, log_name='DA_C')
plot_confusion_matrix(base_model, title='Original test samples for Model C')
torch.cuda.empty_cache()  # Clear GPU memory

In [ ]:
# Execute Model C

base_model, mapping_dict = load_model_variant(
    name='E',
    n_classes=4,
    mapping_key='c',
    model_folder='../models',
    model_filename='C_LCU_2023-04-21-1335.pth' # fix this
)

dataloaders = setup_dataloaders(data_key='c', preprocess_func=base_model.preprocessing_func)
base_model = train_and_evaluate_model(base_model, dataloaders, log_name='DA_C')
plot_confusion_matrix(base_model, title='Original test samples for Model C')
torch.cuda.empty_cache()  # Clear GPU memory